# WISE2MBH Tutorial

In [1]:
import wise2mbh as wm
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import random
no_data = 9876543

In [2]:
allwise = Table.read('../samples/AllWISE_sample.fits')
allwise

ONAME,RA,DEC,Z,T,NED_TYPE,AllWISE,RAJ2000,DEJ2000,eeMaj,eeMin,eePA,W1mag,W2mag,W3mag,W4mag,Jmag,Hmag,Kmag,e_W1mag,e_W2mag,e_W3mag,e_W4mag,e_Jmag,e_Hmag,e_Kmag,ID,ccf,ex,var,qph,pmRA,e_pmRA,pmDE,e_pmDE,d2M,angDist
bytes107,float64,float64,float64,float64,bytes7,bytes19,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,bytes4,bytes1,bytes4,bytes4,int32,int32,int32,int32,float32,float64
WISEAJ112428.14-363548.5,171.1172539,-36.5968225,0.2,9876543.0,Unknown,J112428.14-363548.5,171.1172539,-36.5968225,0.0427,0.0415,130.3,13.449,12.486,9.394,6.977,15.185,14.474,14.293,0.025,0.024,0.033,0.084,0.056,0.061,0.075,1716036401351015875,0000,0,1122,AAAA,160,54,42,55,2.073,1e-06
WISEAJ144229.68-435343.2,220.6236974,-43.8953408,0.3,9876543.0,Unknown,J144229.68-435343.2,220.6236974,-43.8953408,0.1285,0.1177,11.9,15.975,14.663,10.911,7.801,--,--,--,0.055,0.06,0.118,0.195,--,--,--,2205044001351034712,0000,0,011n,AABB,412,466,-34,505,--,1e-06
WISEAJ163728.75+290335.9,249.3698107,29.0599733,0.2,9876543.0,Unknown,J163728.75+290335.9,249.3698107,29.0599733,0.0692,0.0643,4.9,15.637,13.775,9.703,7.347,--,--,--,0.04,0.034,0.038,0.124,--,--,--,2485128701351042160,0000,0,0011,AAAB,-116,295,-250,278,--,2e-06
WISEAJ184321.55-621724.8,280.8398056,-62.2902233,0.2,9876543.0,Unknown,J184321.55-621724.8,280.8398056,-62.2902233,0.0722,0.0689,38.8,14.624,13.762,11.04,8.519,--,--,--,0.036,0.035,0.106,--,--,--,--,2810062101351018362,0000,0,nnnn,AAAU,--,--,--,--,--,5e-06
WISEAJ183203.33+633041.5,278.0139066,63.5115391,0.3,9876543.0,Unknown,J183203.33+633041.5,278.0139066,63.5115391,0.0662,0.0627,129.3,15.811,14.794,11.508,8.357,--,--,--,0.032,0.035,0.086,0.118,--,--,--,2774163601351028837,0000,0,11nn,AAAB,155,233,-229,248,--,7e-06
WISEAJ000517.26-341452.8,1.321926,-34.2480098,0.3,9876543.0,Unknown,J000517.26-341452.8,1.321926,-34.2480098,0.0752,0.0713,141.6,15.1,14.143,10.839,7.887,--,--,--,0.035,0.04,0.114,0.189,--,--,--,18034901351061482,0000,0,110n,AABB,289,256,-282,261,--,7e-06
WISEAJ041514.13-810745.9,63.8088863,-81.1294387,0.2,9876543.0,Unknown,J041514.13-810745.9,63.8088863,-81.1294387,0.0457,0.0449,162.3,14.499,13.543,10.429,7.558,--,--,--,0.025,0.026,0.048,0.099,--,--,--,681081801351062155,0000,0,110n,AAAA,260,66,92,69,--,8e-06
WISEAJ150206.92+781622.6,225.5288639,78.2729584,0.3,9876543.0,Unknown,J150206.92+781622.6,225.5288639,78.2729584,0.0886,0.0831,95.1,15.959,15.085,12.15,9.497,--,--,--,0.037,0.05,0.202,0.512,--,--,--,2232178701351000385,0000,0,11nn,AABC,-761,305,100,285,--,9e-06
WISEAJ050357.41-282633.0,75.989223,-28.4425115,0.2,9876543.0,Unknown,J050357.41-282633.0,75.989223,-28.4425115,0.1182,0.0847,153.4,15.102,14.292,11.284,8.529,--,--,--,0.045,0.045,0.125,0.332,--,--,--,754028801351050034,0000,0,nnnn,AABB,--,--,--,--,--,1.1e-05


Clening process of the sample from AllWISE

In [3]:
size_before_depure = len(allwise)
print(f'Size before depure: {size_before_depure}')

allowed_z = allwise[allwise['Z']!=0]                                                            #Masking of null redshifts
qph_list = allwise['qph'].tolist()                                                              #Extract quality flags
null_phot = ['X', 'Z']                                                                          #Null and uplim values in W bands
not_null_quality = [not any(letter in qph[:-1] for letter in null_phot) for qph in qph_list]    #Test quality flags

allwise = allowed_z[not_null_quality].copy()                                                    #Keep non-null values in quality

size_after_depure = len(allwise)
print(f'Size after depure: {size_after_depure}')

Size before depure: 77712
Size after depure: 77687


Detections with 'U' flag in qph do not contain errors on the WISE bands, so better to set them to a single value, in this case we'll set them to 0

In [4]:
allwise['e_W1mag'] = allwise['e_W1mag'].filled(0)                 #Replace null errors in WISE bands
allwise['e_W2mag'] = allwise['e_W2mag'].filled(0)
allwise['e_W3mag'] = allwise['e_W3mag'].filled(0)

Since this algorithm have a Monte Carlo implementation for error propagation, we recommend to take slices for big sample (rows>200), so for this tutorial, we'll use a random slice from the AllWISE sample.

In [5]:
random_indices = random.sample(range(len(allwise)), 1000)
new_allwise = allwise[random_indices]
new_allwise

ONAME,RA,DEC,Z,T,NED_TYPE,AllWISE,RAJ2000,DEJ2000,eeMaj,eeMin,eePA,W1mag,W2mag,W3mag,W4mag,Jmag,Hmag,Kmag,e_W1mag,e_W2mag,e_W3mag,e_W4mag,e_Jmag,e_Hmag,e_Kmag,ID,ccf,ex,var,qph,pmRA,e_pmRA,pmDE,e_pmDE,d2M,angDist
bytes107,float64,float64,float64,float64,bytes7,bytes19,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,bytes4,bytes1,bytes4,bytes4,int32,int32,int32,int32,float32,float64
SDSS856884946675984384,125.68094,41.66647,0.2110126,-5.0,Galaxy,J082243.42+413959.2,125.680938,41.6664696,0.0509,0.0497,145.6,13.517,13.242,12.498,9.128,15.739,15.079,14.318,0.026,0.029,0.0,--,0.096,0.105,0.084,1253142401351002519,0000,1,11nn,AAUU,258,78,355,88,0.153,0.005579
SDSSJ083232.71+463155.6,128.1363,46.5321314,0.28457433,9876543.0,Unknown,J083232.71+463155.7,128.1363009,46.5321447,0.0788,0.075,167.4,14.877,14.634,12.184,8.933,16.904,15.901,15.248,0.032,0.056,0.0,--,0.204,0.218,0.159,1287146901351015768,0000,0,00nn,AAUU,-206,200,-322,212,0.553,0.048025
SDSSJ090924.39+232526.1,137.351631,23.4239195,0.49248335,9876543.0,Unknown,J090924.39+232526.1,137.3516259,23.4239264,0.1053,0.098,176.4,15.269,15.102,12.284,8.6,--,--,--,0.039,0.083,0.0,--,--,--,--,1368122701351051635,0000,0,11nn,AAUU,218,298,-31,331,--,0.030017
SDSS1074265506761959424,182.87904,61.14136,0.123154365,-5.0,Galaxy,J121130.96+610828.9,182.8790289,61.1413703,0.0485,0.0472,146.5,13.432,13.257,11.884,8.853,15.198,14.48,14.005,0.025,0.028,0.205,--,0.063,0.076,0.078,1830160501351058102,0000,1,00nn,AABU,-83,70,-4,73,0.091,0.041772
SDSS2674067867338565632,156.33562,19.27336,0.13611469,9876543.0,Galaxy,J102520.54+191624.0,156.3356222,19.2733542,0.0585,0.0572,155.6,14.046,13.846,12.513,9.06,15.706,15.074,14.542,0.028,0.036,0.469,--,0.061,0.072,0.086,1568119601351015425,0000,0,00nn,AACU,-29,108,106,117,0.103,0.022261
SDSS3100819674045114368,232.13924,12.45095,0.13263583,9876543.0,Galaxy,J152833.42+122703.4,232.1392514,12.4509581,0.0582,0.0568,168.2,14.091,13.951,12.413,9.304,15.737,15.118,14.42,0.027,0.038,0.341,0.482,0.064,0.089,0.061,2317112101351045404,0000,0,00nn,AABC,55,105,-2,112,0.127,0.049366
J114600.84-260823.3,176.5035356,-26.1398166,0.3,9876543.0,Unknown,J114600.84-260823.3,176.5035225,-26.1398177,0.0618,0.0594,173.0,14.481,13.59,11.586,8.426,--,--,--,0.029,0.034,0.215,--,--,--,--,1766025801351019227,0000,0,53nn,AABU,0,164,0,177,--,0.042477
SDSSJ161532.79+070000.2,243.886665,7.0000598,0.29856867,9876543.0,Unknown,J161532.79+070000.1,243.8866662,7.0000555,0.0903,0.0843,7.3,15.089,14.928,12.061,8.965,17.425,16.05,15.293,0.035,0.074,0.0,0.487,0.265,0.187,0.181,2440107501351007619,0000,0,01nn,AAUC,321,245,580,272,0.45,0.016071
SDSS1774528812755740672,244.38408,27.19927,0.1279412,-5.0,Galaxy,J161732.18+271157.4,244.384086,27.1992779,0.0457,0.0445,135.0,13.145,12.956,10.904,8.562,15.143,14.369,13.894,0.024,0.027,0.102,--,0.069,0.071,0.063,2450127201351035617,dd00,1,110n,AAAU,87,76,-107,76,0.125,0.034278


First, store the observed colors and then define the Monte Carlo arrays for W1, W2 and W3, with the same colors but now for Monte Carlo arrays

In [6]:
new_allwise['W1-W2_obs'] = new_allwise['W1mag'] - new_allwise['W2mag']
new_allwise['W2-W3_obs'] = new_allwise['W2mag'] - new_allwise['W3mag']

In [7]:
w1 = wm.array_montecarlo(new_allwise['W1mag'],new_allwise['e_W1mag'])
w2 = wm.array_montecarlo(new_allwise['W2mag'],new_allwise['e_W2mag'])
w3 = wm.array_montecarlo(new_allwise['W3mag'],new_allwise['e_W3mag'])
z = np.array(new_allwise['Z'])

w1w2_obs = w1-w2
w2w3_obs = w2-w3

# K-corrections for the defined sample

Now that all initial colors are created, one can start by the first masking of the reject zone by deleting all sources with a W2-W3>4.4

In [8]:
first_reject_zone = new_allwise['W2-W3_obs']<=4.4

In [9]:
w1 = w1[first_reject_zone]
w2 = w2[first_reject_zone]
w3 = w3[first_reject_zone]
z = z[first_reject_zone]

w1w2_obs = w1w2_obs[first_reject_zone]
w2w3_obs = w2w3_obs[first_reject_zone]

new_allwise = new_allwise[first_reject_zone]

It is necessary to create new arrays to store K-corrected W1abs and the K-correction factor of W1-W2 and W2-W3

In [10]:
w1w2_kcors = np.zeros(np.shape(w1w2_obs)[0])                                                
w2w3_kcors = np.zeros(np.shape(w2w3_obs)[0])
w1abs = np.zeros(np.shape(w1))

In [11]:
object_condition = (new_allwise['NED_TYPE']=='RadioS') | (new_allwise['NED_TYPE']=='QSO')
color_condition_1 = (new_allwise['W1-W2_obs']>0.8) & (new_allwise['W2-W3_obs']<2.2)
color_condition_2 = (new_allwise['W1-W2_obs']>wm.w1w2_treshold_qso(new_allwise['W2-W3_obs'])) & (new_allwise['W2-W3_obs']>=2.2) & (new_allwise['W2-W3_obs']<=4.4)

suboptimal_cond = (new_allwise['Z']>=0.5) & (new_allwise['Z']<=3) & ~object_condition & ~(color_condition_1 | color_condition_2)
nok_cond = (new_allwise['Z']>3) | object_condition | color_condition_1 | color_condition_2

optimal_sample = new_allwise[(new_allwise['Z']<0.5) & ~object_condition & ~(color_condition_1 | color_condition_2)]
suboptimal_sample = new_allwise[suboptimal_cond]                                                                            #Samples for k-correcion                
nok_sample = new_allwise[nok_cond]

To create the final quality tag, here we store the K-correction tag

In [12]:
new_allwise['K_QUALITY'] = int(0)

new_allwise['K_QUALITY'] = np.where(suboptimal_cond, int(1), new_allwise['K_QUALITY'])
new_allwise['K_QUALITY'] = np.where(nok_cond, int(2), new_allwise['K_QUALITY'])

Now we import the look-up tables from Jarrett+2023 to use

In [13]:
e_table = wm.kcorr_table('E')
l_table = wm.kcorr_table('L')
s_table = wm.kcorr_table('S')

In [14]:
e_table

,z,f1,W1-W2,W1-W3,W3-W4,W2-W3
0,0.00050,1.000,-0.000,-0.000,-0.000,-0.000
1,0.00100,1.000,0.000,0.001,0.000,0.001
2,0.00150,0.998,0.002,0.001,0.001,-0.001
3,0.00200,0.995,0.000,0.000,0.001,0.000
4,0.00250,0.995,0.005,0.002,0.002,-0.003
...,...,...,...,...,...,...
827,2.87603,0.043,0.930,1.733,-0.180,0.803
828,2.90479,0.043,0.941,1.758,-0.180,0.817
829,2.93384,0.043,0.952,1.783,-0.181,0.831
830,2.96318,0.043,0.963,1.808,-0.182,0.845


In [15]:
identifier = np.random.choice(np.arange(10), size=3, replace=False)
for idn, sample in zip(identifier,[optimal_sample, suboptimal_sample, nok_sample]):
    if len(sample)!=0:
        if idn!=identifier[2]:                                                                      #Color k-correction for sample with z<0.5
            object_condition = (sample['NED_TYPE']=='RadioS') | (sample['NED_TYPE']=='QSO')

            e_index = np.where((sample['T']<=-3) & ~object_condition)[0]
            l_index = np.where((sample['T']>-3) & (sample['T']<=0) & ~object_condition)[0]          #Divide samples in Elliptical, Lenticular and Spiral by T value
            s_index = np.where((sample['T']>0) & (sample['T']!=no_data) & ~object_condition)[0]

            no_type_index = np.where((sample['T']==no_data) & ~object_condition)[0]                 #Sample without T values
            w1w2_no_type = sample[no_type_index]['W1-W2_obs']                                      
            w2w3_no_type = sample[no_type_index]['W2-W3_obs']  
            z_no_type = sample[no_type_index]['Z']                                                    

            e_w1_kcorrected = wm.w1_k_corrected(lookup_table=e_table,w1=w1[e_index],z=z[e_index])
            l_w1_kcorrected = wm.w1_k_corrected(lookup_table=l_table,w1=w1[l_index],z=z[l_index])   #W1 k-corrected 
            s_w1_kcorrected = wm.w1_k_corrected(lookup_table=s_table,w1=w1[s_index],z=z[s_index])

            e_w2w3_kcor = wm.get_correction_factor(lookup_table=e_table, redshift=sample[e_index]['Z'], correction_factor='W2-W3')
            l_w2w3_kcor = wm.get_correction_factor(lookup_table=l_table, redshift=sample[l_index]['Z'], correction_factor='W2-W3')     #W2-W3 k-correction factor
            s_w2w3_kcor = wm.get_correction_factor(lookup_table=s_table, redshift=sample[s_index]['Z'], correction_factor='W2-W3')

            e_w1w2_kcor = wm.get_correction_factor(lookup_table=e_table, redshift=sample[e_index]['Z'], correction_factor='W1-W2')
            l_w1w2_kcor = wm.get_correction_factor(lookup_table=l_table, redshift=sample[l_index]['Z'], correction_factor='W1-W2')     #W1-W2 k-correction factor
            s_w1w2_kcor = wm.get_correction_factor(lookup_table=s_table, redshift=sample[s_index]['Z'], correction_factor='W1-W2')

            #Both k-correction factors for no type sample, using division by observed W2-W3 color
            factor_w2 = 171.787
            factor_w3 = 31.674

            w2w3_limit = -2.5*(np.log10(factor_w3/factor_w2) + 0.1)                             #Last value is the flux limit from Mateos et al. (2012)

            no_type_w1w2_kcor = [
                wm.get_correction_factor(lookup_table=e_table, redshift=z, correction_factor='W1-W2') if (color_x<=w2w3_limit)
                else wm.get_correction_factor(lookup_table=s_table, redshift=z, correction_factor='W1-W2') if (color_x>w2w3_limit) else 0
                for color_x,color_y,z in zip(w2w3_no_type,w1w2_no_type, z_no_type)
                ]
            no_type_w2w3_kcor = [
                wm.get_correction_factor(lookup_table=e_table, redshift=z, correction_factor='W2-W3') if (color_x<=w2w3_limit)
                else wm.get_correction_factor(lookup_table=s_table, redshift=z, correction_factor='W2-W3') if (color_x>w2w3_limit)  else 0
                for color_x,color_y,z in zip(w2w3_no_type,w1w2_no_type, z_no_type)
                ]

            no_type_w1_kcorrected = [
                wm.w1_k_corrected(lookup_table=e_table,w1=w1[no_type_id],z=np.array([z]))[0] if (color_x<=w2w3_limit)
                else wm.w1_k_corrected(lookup_table=s_table,w1=w1[no_type_id],z=np.array([z]))[0] if (color_x>w2w3_limit) else w1[no_type_index]
                for color_x,color_y,z,no_type_index in zip(w2w3_no_type,w1w2_no_type, z_no_type, no_type_index)
                ]   

            e_w1abs_kcorrected = e_w1_kcorrected - wm.distance_modulus_z(sample[e_index]['Z'])[:,None]          #W1 k-correction for e_sources
            l_w1abs_kcorrected = l_w1_kcorrected - wm.distance_modulus_z(sample[l_index]['Z'])[:,None]          #W1 k-correction for l_sources
            s_w1abs_kcorrected = s_w1_kcorrected - wm.distance_modulus_z(sample[s_index]['Z'])[:,None]          #W1 k-correction for s_sources
            no_type_w1abs_kcorrected = no_type_w1_kcorrected - wm.distance_modulus_z(dist[z_no_type])[:,None]   #W1 k-correction for no_type_sources

            if np.shape(no_type_w1abs_kcorrected)==(0,0):
                no_type_w1abs_kcorrected = no_type_w1abs_kcorrected.reshape(0,1000)

            w1w2_kcors[e_index] = e_w1w2_kcor
            w1w2_kcors[l_index] = l_w1w2_kcor                   #W1-W2 k-correctrion factors filled
            w1w2_kcors[s_index] = s_w1w2_kcor
            w1w2_kcors[no_type_index] = no_type_w1w2_kcor

            w2w3_kcors[e_index] = e_w2w3_kcor
            w2w3_kcors[l_index] = l_w2w3_kcor                   #W2-W3 k-correctrion factors filled
            w2w3_kcors[s_index] = s_w2w3_kcor
            w2w3_kcors[no_type_index] = no_type_w2w3_kcor

            w1abs[e_index] = e_w1abs_kcorrected
            w1abs[l_index] = l_w1abs_kcorrected                 #W1abs k-corrected filled
            w1abs[s_index] = s_w1abs_kcorrected
            w1abs[no_type_index] = no_type_w1abs_kcorrected
            
        else:
            w1abs_kcorrected = w1[np.where(nok_cond)[0]] - wm.distance_modulus_z(sample[np.where(nok_cond)[0]]['Z'])[:,None]
            w1abs[np.where(nok_cond)[0]] = w1abs_kcorrected

            w1w2_kcors[np.where(nok_cond)[0]] = 0
            w2w3_kcors[np.where(nok_cond)[0]] = 0                                        #For sources with z>3, no k-correction is applied
    else:
        continue

TypeError: 'numpy.float64' object is not iterable

In [18]:
wm.get_correction_factor(lookup_table=e_table,redshift=z_no_type[0])

TypeError: 'numpy.float64' object is not iterable

In [20]:
np.float64

numpy.float64